### Code for the paper "Negotiating Comfort: Simulating Personality-Driven LLM Agents in Shared Residential Social Networks"

In this study, we use generative agents to model human decisions in the context of building energy modeling. We use Crowd framework for social network simulations and execute our code on Google Colab using an L4 GPU. 

  ##### Simulation scenario:
  In a building with 34 apartments and a central heating and cooling system, each day: 
  1. In the first stage, families come together to decide on a temperature for the system. Each member suggests a temperature, given their temperature preferences, their family members' preferences, and the degree outside.
  2. Average of all members' choices is stored as *family's choice* and is given to the Family Representative. 
  3. In the second stage, Family Representatives make another decision which they submit to the building poll. In this decision, besides the previous information, the representatives are informed about their friends: Their closeness level, friend's family's degree choice for that day, and friend's degree suggestions for the last three days. Agents provide a final degree decision, and are given the option to update their closeness levels with friends depending on the alignment of their decisions.
  4. Average of the temperatures submitted to the building poll is set as the final building temperature, and is stored as *temperature set*. 

  Meanwhile, each member is also assigned a *happiness value* initialized to 100, and is updated each day with respect to their previous day's choice and the final temperature set in the building. 


In [ ]:
# Step 1: Mount Google Drive to access 'crowd' library
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Step 2: Set up paths and import your Crowd library
import sys
%cd /content/drive/My Drive/Crowd_Related_Work/wheel

In [ ]:
# Step 3: Install Crowd and other needed libraries
!pip install crowd-0.9.0-py3-none-any.whl

In [ ]:
!pip install names_dataset # used to assign names to nodes
!pip install bitsandbytes # used for quantization
!pip install pvlib # used to get the weather data

In [ ]:
# Change directory to the parent folder so that we will have the projects folder there
%cd /content/drive/My Drive/Crowd_Related_Work/

In [ ]:
# Step 4: Import the libraries we will use
import json
import os
import random
import time
from names_dataset import NameDataset
import networkx as nx
import numpy as np
import pandas as pd
import math
import pvlib
import re
from crowd.project_management.project import Project
from crowd.egress.file_egress import file_egress as fd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from huggingface_hub import login
login(token="your_hf_token")

In [ ]:
# Step 5: Define helper methods

# generate_names method directly taken from: GABM-Epidemic
# https://github.com/bear96/GABM-Epidemic/blob/main/utils.py#L18
def generate_names(n: int, s: int, country_alpha2='US'):
    '''
    Returns random names as names for agents from top names in the USA
    Used in World.init to initialize agents
    '''

    # This function will randomly selct n names (n/2 male and n/2 female) without
    # replacement from the s most popular names in the country defined by country_alpha2
    if n % 2 == 1:
        n += 1
    if s % 2 == 1:
        s += 1

    nd = NameDataset()
    male_names = nd.get_top_names(s//2, 'Male', country_alpha2)[country_alpha2]['M']
    female_names = nd.get_top_names(s//2, 'Female', country_alpha2)[country_alpha2]['F']
    if s < n:
        raise ValueError(f"Cannot generate {n} unique names from a list of {s} names.")
    # generate names without repetition
    names = random.sample(male_names, k=n//2) + random.sample(female_names, k=n//2)
    del male_names
    del female_names
    random.shuffle(names)
    return names

# Adds names as node parameters to the networkX object
def add_name_parameter(graph):
    names = generate_names(100, 200)
    attr = {}
    for n in graph.nodes():
        selected_name = random.choice(names)
        attr.update({n: {"name": selected_name}})
        names.remove(selected_name)

    nx.set_node_attributes(graph, attr)

# Adds traits, last 3 temperature suggestions, node id, and node type as node parameters to the networkX object
def add_trait_parameters(graph, positive_percentage = 50):
  attr = {}

  for n in graph.nodes():
    generated_traits = generate_traits(positive_percentage)
    generated_traits.update({'last_3_suggestions' : [None, None, None]})
    generated_traits.update({'node' : n}) 
    generated_traits.update({'node_type' : "Family_Rep"}) 
    attr.update({n: generated_traits})

  nx.set_node_attributes(graph, attr)

# Used to change the positive percentage distribution of the nodes without changing any other node parameters
def update_traits(graph, positive_percentage = 50):
  attr = {}

  for n in graph.nodes():
    generated_traits = generate_traits(positive_percentage)
    attr.update({n: generated_traits})

  nx.set_node_attributes(graph, attr)

# Using Mistral7B Instruct model from Hugging Face, tokenizes the input, sends the query to LLM, generates an output and returns the decoded output
def get_completion_from_messages(model, tokenizer, user_prompt, max_tokens=250, temperature=0.1):
    try:
        # Tokenize the input with padding
        inputs = tokenizer(user_prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

        # Generate text with attention mask and padding token set
        outputs = model.generate(
            inputs.input_ids,
            max_new_tokens=max_tokens,
            temperature=temperature,
            do_sample=True,
            attention_mask=inputs["attention_mask"],
            pad_token_id= tokenizer.eos_token_id,  # Ensure the padding is handled
        )

        # Remove the input part from the output 
        outputs = outputs[:, inputs.input_ids.shape[-1]:]

        # Decode the generated tokens to return the text
        return tokenizer.decode(outputs[0], skip_special_tokens=True)

    except Exception as e:
        print(f"Error generating text: {e}")
        return None

In [ ]:
# Defining sets of traits that should not co-occur conceptually
CONFLICTS = [
    {"Selfless", "Uncooperative"},
    {"Environmentalist", "Wasteful"},
    {"Impulsive", "Unemotional"}
]

# Helper method to determine if this set of traits has any conflicts
def has_conflict(trait_dict, conflict_sets):
    values = set(trait_dict.values())
    for conflict in conflict_sets:
        if conflict.issubset(values):
            return True
    return False

# Function to generate traits
def generate_traits(positive_percentage, max_attempts = 20):

  # Define traits for each category
  traits = {
      "N2": {
          "positive": "Easygoing",
          "negative": "Easily-angered"
      },
      "N5": {
          "positive": "Self-controlled",
          "negative": "Impulsive"
      },
      "E3": {
          "positive": "Assertive",
          "negative": "Passive"
      },
      "O3": {
          "positive": "Emotional",
          "negative": "Unemotional"
      },
      "A3": {
          "positive": "Selfless",
          "negative": "Selfish"
      },
      "A4": {
          "positive": "Cooperative",
          "negative": "Uncooperative"
      },
      "C6": {
          "positive": "Cautious",
          "negative": "Careless"
      },
      "environmentalism": {
          "positive": "Environmentalist",
          "negative": "Not environmentalist"
      },
      "frugality": {
          "positive": "Frugal",
          "negative": "Wasteful"
      }
  }
  # Ensure the positive_percentage provided by the user is between 0 and 100
  if not (0 <= positive_percentage <= 100):
    raise ValueError("positive_percentage must be between 0 and 100")

  # For at most max_attempts times, assign traits and reassign if there is a conflict
  for attempt in range(max_attempts):
        result = {}

        # Calculate the number of positive and negative traits to pick
        positive_count = int(len(traits) * (positive_percentage / 100))
        negative_count = len(traits) - positive_count

        # Get a list of categories
        categories = list(traits.keys())

        # Shuffle the categories for randomness
        random.shuffle(categories)

        # Select positive traits first
        for category in categories[:positive_count]:
            result[category] = traits[category]["positive"]

        # Select negative traits for the remaining categories
        for category in categories[positive_count:]:
            result[category] = traits[category]["negative"]

        # Check for conflicts
        if not has_conflict(result, CONFLICTS):
            return result

  return result

In [ ]:
# Methods to get weather data from pvlib
def get_daily_avg_temperature(latitude, longitude, tz='UTC', location_name='Unknown'):
    """
    Fetches daily average air temperatures using PVGIS TMY data.
    No hourly split — just 24h average per day.

    Returns:
    - pandas DataFrame with 'date' and 'daily_avg_temp'
    """
    try:
        # Define location
        location = pvlib.location.Location(latitude, longitude, tz=tz, name=location_name)

        # Get TMY data
        tmy_data, _, _, metadata = pvlib.iotools.get_pvgis_tmy(
            latitude=latitude,
            longitude=longitude,
            map_variables=True
        )


        # Resample to daily mean
        daily_avg = tmy_data['temp_air'].resample('D').mean().rename('daily_avg_temp')
        df = daily_avg.to_frame()

        # Remove rows where daily_avg_temp is empty or NaN
        df = df.dropna(subset=['daily_avg_temp'])
        df['daily_avg_temp'] = df['daily_avg_temp'].round().astype(int)

        # Convert index to a column
        df = df.reset_index()
        df.rename(columns={'index': 'time(UTC)'}, inplace=True)

        # Change all years to 2014
        df['time(UTC)'] = df['time(UTC)'].apply(lambda dt: dt.replace(year=2014))

        # Sort by date
        df = df.sort_values('time(UTC)')

        # Remove time part (keep only the date)
        df['time(UTC)'] = df['time(UTC)'].dt.date

        # Reset index
        df = df.reset_index(drop=True)
        
        # Save the output to a csv file (optional)
        # df.to_csv(f'{location_name}_tmy_daily_avg3.csv')
        return df

    except Exception as e:
        print(f"Failed to retrieve daily temperatures for {location_name}: {e}")
        return None

def get_season_weather_daily(weather_df, season_name):
    """
    Filters daily weather DataFrame by season (summer or winter).
    Input: DataFrame from get_daily_avg_temperature()
    Returns: filtered DataFrame
    """
    if season_name.lower() == 'winter':
        months = [12, 1, 2]
    elif season_name.lower() == 'summer':
        months = [6, 7, 8]
    else:
        raise ValueError("Season name must be 'winter' or 'summer'.")

    #Extract month from 'time(UTC)' column (which is of type datetime.date)
    season_df = weather_df[weather_df['time(UTC)'].apply(lambda d: d.month).isin(months)]

    # Reset index
    season_df = season_df.reset_index(drop=True)

    return season_df

from datetime import datetime

def get_weather_between_dates(weather_df, start_str, end_str, date_format="%d/%m"):
    """
    Filters daily weather DataFrame by a specific date range (inclusive).

    Parameters:
    - weather_df: DataFrame from get_daily_avg_temperature()
    - start_str: start date as string (e.g., '15/02')
    - end_str: end date as string (e.g., '15/03')
    - date_format: format of the input date strings (default is '%d/%m')

    Returns:
    - Filtered DataFrame with rows between the specified dates.
    """
    try:
        # Convert start and end to datetime.date with dummy year, 2014
        start_date = datetime.strptime(start_str, date_format).replace(year=2014).date()
        end_date = datetime.strptime(end_str, date_format).replace(year=2014).date()

        # Filter by date range (inclusive)
        filtered_df = weather_df[
            (weather_df['time(UTC)'] >= start_date) &
            (weather_df['time(UTC)'] <= end_date)
        ].reset_index(drop=True)

        return filtered_df
    except Exception as e:
        print(f"Failed to filter weather data between {start_str} and {end_str}: {e}")
        return None


In [ ]:
# Step 6: Extract the daily average temperatures from the typical meteorological year
# between Feb 15 and Mar 16 
ankara_daily = get_daily_avg_temperature(
    latitude=39.9334,
    longitude=32.8597,
    tz='Europe/Istanbul',
    location_name='Ankara'
)
mid_feb_to_mid_mar_df = get_weather_between_dates(ankara_daily, '15/02', '16/03')

In [ ]:
# Step 7: LLM setup for inference
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define configuration for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit_fp32_cpu_offload=True
)

# Load the model with quantization and a manual device map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # Use quantization for 8-bit loading
    device_map="auto"  # Automatically allocate layers to devices
)

# Now you can proceed with using the model for inference
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.eos_token_id

In [ ]:
# Step 7.5: Other helper methods for the simulation

# Taking the Karate Club network of 34 nodes as an input, attach newly created family members to each node
# to form fully-connected subgraphs representing the "family" relations
def create_family(network, family_count, sample_family_sizes):
  prev_G_size = network.number_of_nodes()
  agent_id =  prev_G_size # we will start giving IDs following the last id
  family_sizes = []
  agents_to_add = 0
  families = {}

  for i in range(family_count):
    family_size = random.choice(sample_family_sizes)
    agents_to_add += family_size
    family_sizes.append(family_size)

  names = generate_names(agents_to_add, agents_to_add)

  for family_id, size in enumerate(family_sizes):
    members = []
    for _ in range(size):
      curr_traits = generate_traits(50)
      network.add_node(
        agent_id,
        node = family_id, 
        happiness_level = 100,
        degree_choice = 100,
        heater_preference = random.choice(["hot", "warm", "neutral", "cool", "cold"]),
        N2 = curr_traits["N2"],
        N5 = curr_traits["N5"],
        E3 = curr_traits["E3"],
        O3 = curr_traits["O3"],
        A3 = curr_traits["A3"],
        A4 = curr_traits["A4"],
        C6 = curr_traits["C6"],
        environmentalism = curr_traits["environmentalism"],
        frugality = curr_traits["frugality"],
        age = random.randrange(18,65),
        node_type = "Family_Member", 
        name = names[agent_id - prev_G_size],
        last_3_suggestions = [None, None, None]
      )
      members.append(agent_id)
      agent_id += 1

    # Connect all agents in the same family
    members.append(family_id) # Add the family representative to family
    for i in range(size + 1): # Plus one for newly added member
      for j in range(i + 1, size + 1):
        network.add_edge(members[i], members[j], relation = "family")

    # Add to families dict
    families[family_id] = members

  return families

In [ ]:
# Before iteration methods
# Family-level
def decide_degree_for_families(network, families, model, tokenizer):

  # First call decide degree
  for family_id, members in families.items():
    for member in members:
      ask_agent_family_level(network, member, model, tokenizer)

  # Now that everyone has decided degrees, take avg and set representative's "family-request" parameter
  for family_id, members in families.items():
    set_family_degree(network, family_id, members)

In [ ]:
# Building level
def decide_degree_for_building(network, families, model, tokenizer):
  degrees_total = 0
  reps_count = 0
  # First call decide degree and get total
  for agent_id in network.G.nodes():
    if network.G.nodes[agent_id]['node_type'] == "Family_Rep":
      degrees_total += decide_degree_with_friends(network, agent_id, families, model, tokenizer)
      reps_count += 1

  if reps_count != 0:
    avg_degree = int(round(degrees_total/reps_count))

  network.G.graph["temperature_set"] = avg_degree
  # return avg_degree



In [ ]:
# Building-level helper methods
def decide_degree_with_friends(network, agent_id, families, model, tokenizer):
  neighbors = list(network.G.neighbors(agent_id))
  friends = []

  # get a list of friends
  for n in neighbors:
    if n!= agent_id and network.G.nodes[n]['node_type'] == "Family_Rep":
      friends.append(n)

  # call decide degree
  return ask_agent_building_level(network, agent_id, len(families[agent_id]), model, tokenizer, friends)

def ask_agent_building_level(network, agent_id, family_member_count, model, tokenizer, friends):
  reasoning, degree_choice, updated_closeness = get_building_level_deg(network, agent_id, model, tokenizer, friends, family_member_count)

  if reasoning is None:
    print(f"Reasoning of {agent_id} was None.")

  try:
    degree_choice = int(float(degree_choice))
  except:
    print(f"Degree choice of {agent_id} is not valid. Defaulting with 21.\nLLM's answer was '{degree_choice}'")
    degree_choice = 21

  # Update this family representative's degree choice
  network.G.nodes[agent_id]['degree_choice'] = degree_choice

  # Apply the closeness updates
  for key, new_weight in updated_closeness.items():
    # This matches the name(ID), ex: Clara(27)
    match = re.match(r"(.+)\((\d+)\)", key)
    # Ex:
    # match.group(1) = "Clara"
    # match.group(2) = "27"
    if match:
        friend_id = int(match.group(2))
        if network.G.has_edge(agent_id, friend_id):
            network.G.edges[agent_id, friend_id]['weight'] = int(new_weight)
        elif network.G.has_edge(friend_id, agent_id):  # undirected case
            network.G.edges[friend_id, agent_id]['weight'] = int(new_weight)

  return degree_choice

def get_friend_info(network, agent_id, friends, family_member_count):

  if (len(friends) == 0):
    return ""

  info = f"""
    You have {len(friends)} friends in this building. You have a closeness level between 1 to 5 with each friend,
    where a higher number indicates a stronger friendship. Below is a summary about your friends:
  """

  friend_list_str = ""
  friend_degree_suggestion_str = ""
  friend_family_decision_str = ""

  for id in friends:
    name = network.G.nodes[id]['name']

    if network.G.has_edge(agent_id, id):
      weight = network.G[agent_id][id]["weight"]
    else:
      weight = network.G[id][agent_id]["weight"]

    friend_list_str += f"""
      - {name} (ID: {id}) - closeness level: {weight}"""

    # starts with the 4th day for the current friend
    if network.G.nodes[id]['last_3_suggestions'][0] is not None:
      friend_degree_suggestion_str += f"""
      - {name}'s last 3 degree suggestions were: {network.G.nodes[id]['last_3_suggestions']}"""

    friend_family_decision_str += f"""
      - Today {name}'s family decided on: {network.G.nodes[id]['family-request']}"""

  # starts with the 4th day for the current agnent
  if network.G.nodes[agent_id]['last_3_suggestions'][0] is not None:
    friend_degree_suggestion_str += f"""
      - Your last 3 suggestions were: {network.G.nodes[agent_id]['last_3_suggestions']}

    """

  if family_member_count < 2:
    friend_family_decision_str += f"""
      - Today, you decided on: {network.G.nodes[agent_id]['family-request']}

    """
  else:
    friend_family_decision_str += f"""
      - Today, your family decided on: {network.G.nodes[agent_id]['family-request']}

    """

  info += friend_list_str
  info += friend_degree_suggestion_str
  info += friend_family_decision_str

  info += f"""
    Note: Your friends also knew your family's decision before submitting the last degree suggestion to the building poll.
  """

  return info

def get_building_level_deg(network, agent_id, model, tokenizer, friends, family_member_count):
    # Generate prompt accordingly and call the Generative AI model

    name = network.G.nodes[agent_id]['name']

    question_prompt = f"""[INST]
        You are {name}. You are {network.G.nodes[agent_id]['age']} years old.

        Your personality traits are:
        - {network.G.nodes[agent_id]['N2']}
        - {network.G.nodes[agent_id]['N5']}
        - {network.G.nodes[agent_id]['E3']}
        - {network.G.nodes[agent_id]['O3']}
        - {network.G.nodes[agent_id]['A3']}
        - {network.G.nodes[agent_id]['A4']}
        - {network.G.nodes[agent_id]['C6']}
        - {network.G.nodes[agent_id]['environmentalism']}
        - {network.G.nodes[agent_id]['frugality']}
        """


    if family_member_count > 1:
      question_prompt += f"""
        You live in an apartment building, sharing a home with other family members. Every day, you come together and decide what you should set the degree of
        the heater."""
    else:
      question_prompt += f"""
        You live alone in an apartment building. Every day, you decide what you should set the degree of the heater."""

    question_prompt += f"""
        {get_friend_info(network, agent_id, friends, family_member_count)}

        Also, your heater preference is {network.G.nodes[agent_id]['heater_preference']}.

        Reference table:
        Cold: <18 degrees
        Cool: 18-20 degrees
        Neutral: 21-24 degrees
        Warm: 25-27 degrees
        Hot: >27 degrees

        Your current happiness level is {network.G.nodes[agent_id]['happiness_level']} (from 1 to 100). Happiness increases when the final heater degree is close to your preference, and decreases otherwise.

        Currently, it is {network.G.graph["degree_outside"]} degrees outside.
      """

    curr_day = network.G.graph["current_day"]

    if family_member_count > 1:
      question_prompt += f"""
        You are your family's representative.

        **Your task**: Decide the best heater degree **today** based on:
        - Your personality traits,
        - Your heater preference
        - Your family's request"""
    else:
      question_prompt += f"""
        **First task**: Decide the best heater degree **today** based on:
        - Your personality traits,
        - Your heater preference"""

    question_prompt += f"""
        - Your friends' choices
        - The current outside temperature

        **Second Task**: You may choose to update your closeness levels to your friends based on:
        - How aligned their decisions in the last 3 days were with yours
        - Your personality traits

        Your choice will be sent to the building poll and the degree will be set depending on everyone's choices.
        Please also provide your reasoning.

         **Response format (must follow strictly):**

        Reasoning: [Explain your reasoning in 1 paragraph. Mention how your traits influence your decision.]
        Degree: [An **integer only**, e.g., 22]
        Updated closeness: {{ "friend_name(id)": new_level, "friend_name(id2)": new_level, ... }}

        - You must use **double quotes** for both keys and string values.
        - The `Updated closeness` must be a valid **JSON object**.
        - New closeness levels must be between [1, 5].

        Only provide **one** reasoning and **one** degree. If multiple reasonings exist, combine them into one paragraph.
        If you update any closeness levels, list them explicitly in the format above.
        Any incorrect formatting will result in your response being ignored.

        Example (format only, not your situation):

        Reasoning: I want to support my family’s request while staying close to my own preference. Clara’s suggestions were far from mine, and I feel she doesn’t care about my comfort, so I will reduce my closeness with her.
        Degree: 20
        Updated closeness: {{ "Clara(27)": 1, "David(40)": 5 }}
        [/INST]"""

    try:
        # print("Prompt for node" , agent_id, "in phase 2:", question_prompt)
        if len(friends) > 10: # increase the max tokens to allow more space for people with more friends
          output = get_completion_from_messages(model = model,
                                              tokenizer = tokenizer,
                                              user_prompt = question_prompt,
                                              max_tokens = 300)
        else:
          output = get_completion_from_messages(model = model,
                                              tokenizer = tokenizer,
                                              user_prompt = question_prompt)
        # print("Output for node", agent_id, ":", output)
    except Exception as e:
        print(f"{e}\nProgram paused. Retrying after 10s...")
        time.sleep(10)
        output = get_completion_from_messages(model = model,
                                              tokenizer = tokenizer,
                                              user_prompt = question_prompt)

    reasoning = ""
    degree_choice = ""
    updated_closeness = {}
    try:
        # Extract reasoning
        reasoning_match = re.search(r"Reasoning:\s*(.+?)\n(?:Degree:|$)", output, re.DOTALL)
        reasoning = reasoning_match.group(1).strip() if reasoning_match else None

        # Extract degree (first integer after "Degree:")
        degree_match = re.search(r"Degree:\s*(\d+)", output)
        degree_choice = degree_match.group(1) if degree_match else None

        # Extract updated closeness (JSON-like block)
        closeness_match = re.search(r"Updated closeness:\s*\{(.*?)\}", output, re.DOTALL)
        closeness_block = closeness_match.group(1).strip() if closeness_match else "{}"
        # print("closeness block:", closeness_block)
        try:
            if closeness_block[0] != "{":
              closeness_json_str = "{" + closeness_block + "}"
            else:
              closeness_json_str = closeness_block
            # print("closeness_json_str:", closeness_json_str)
            updated_closeness = json.loads(closeness_json_str)
            # print("updated closeness:", updated_closeness)
        except Exception as e:
            # print("Error decoding JSON:", e)
            updated_closeness = {}

        # # Print the extracted values
        # print("Reasoning before return in get_building_level_deg:", reasoning)
        # print("Degree choice before return in get_building_level_deg:", degree_choice)

        save_current_agent_response(
            curr_day,
            agent_id,
            question_prompt,
            output,
            reasoning,
            degree_choice,
            happiness_level=None,
            updated_closeness = updated_closeness
          )

    except Exception as e:
        print("Reasoning, degree or closeness values were not parsed correctly:", e)
        print("Output:", output)
        degree_choice = "Error"
        reasoning = None
        updated_closeness = {}

    return reasoning, degree_choice, updated_closeness

In [ ]:
# Family-level helper methods

def ask_agent_family_level(network, agent_id, model, tokenizer):
  reasoning, degree_choice, happiness_level = get_family_deg_and_happiness(network, agent_id, model, tokenizer)

  if reasoning is None:
    print(f"Reasoning of {agent_id} was None.")

  try:
    degree_choice = int(float(degree_choice))
  except:
    print(f"Degree choice of {agent_id} is not valid. Defaulting with 21.\nLLM's answer was '{degree_choice}'")
    degree_choice = 21

  network.G.nodes[agent_id]['degree_choice'] = degree_choice

  try:
    happiness_level = int(happiness_level)
    if happiness_level < 0 or happiness_level > 100:
      happiness_level = network.G.nodes[agent_id]['happiness_level'] # keep the same level
  except:
    print(f"Happiness level of {agent_id} is not valid. It will not be updated in this iteration.\nLLM's answer was '{happiness_level}'")
    happiness_level = network.G.nodes[agent_id]['happiness_level'] # keep the same level

  network.G.nodes[agent_id]['happiness_level'] = happiness_level


def get_family_heater_preferences(network, agent_id):
  preferences = "Heater preferences of you and your family members include:\n"
  current_fam = network.G.nodes[agent_id]["node"]
  fam_members_count = 0
  for n in network.G.nodes():
    if n != agent_id and network.G.nodes[n]["node"] == current_fam:
      name = network.G.nodes[n]['name']
      pref = network.G.nodes[n]['heater_preference']
      preferences += f"""
            - """ + name + """ prefers: """ + pref
      fam_members_count += 1

  return preferences, fam_members_count

def get_family_deg_and_happiness(network, agent_id, model, tokenizer):
    # Generate prompt accordingly and call the Generative AI model

    name = network.G.nodes[agent_id]['name']
    preferences, fam_members_count = get_family_heater_preferences(network, agent_id)

    question_prompt = f"""[INST]
        You are {name}. You are {network.G.nodes[agent_id]['age']} years old.

        Your personality traits are:
        - {network.G.nodes[agent_id]['N2']}
        - {network.G.nodes[agent_id]['N5']}
        - {network.G.nodes[agent_id]['E3']}
        - {network.G.nodes[agent_id]['O3']}
        - {network.G.nodes[agent_id]['A3']}
        - {network.G.nodes[agent_id]['A4']}
        - {network.G.nodes[agent_id]['C6']}
        - {network.G.nodes[agent_id]['environmentalism']}
        - {network.G.nodes[agent_id]['frugality']}
        """

    if fam_members_count > 0:
      question_prompt += f"""
        You live in an apartment building and share a home with other family members. Every day, you come together and decide what you should set the degree of
        the heater. Later, your family representative will submit this degree to the building-wide poll and the final temperature will be set depending on everyone's choices.

        {preferences}
        - You prefer: {network.G.nodes[agent_id]['heater_preference']}
         """
    else:
      question_prompt += f"""
        You live alone in an apartment building. Every day, you decide what you should set the degree of the heater. Later, you will submit this degree to the building poll
        and the final temperature will be set depending on everyone's choices.
        Your heater preference is {network.G.nodes[agent_id]['heater_preference']}.
      """

    curr_day = network.G.graph["current_day"]

    question_prompt += f"""
      Reference table:
        Cold: <18 degrees
        Cool: 18-20 degrees
        Neutral: 21-24 degrees
        Warm: 25-27 degrees
        Hot: >27 degrees

      Your current happiness level is {network.G.nodes[agent_id]['happiness_level']} (from 1 to 100). Happiness increases when the final heater degree is close to your preference, and decreases otherwise.

      Currently, it is {network.G.graph["degree_outside"]} degrees outside.

       **First task**: Decide the best heater degree **today** based on:
        - Your personality traits,"""

    if fam_members_count > 0:
        question_prompt += f"""
        - Heater preference of you and your family,"""
    else:
        question_prompt += f"""
        - Your heater preference,"""

    question_prompt += f"""
        - The current outside temperature,
        - Your current happiness level.

    """

    # Add this only after the first epoch
    if network.G.nodes[agent_id]['degree_choice'] < 100:
        question_prompt += f"""
            **Second task**: Set a new happiness for yourself (an integer from 1 to 100) based on:
            - The building-wise degree set yesterday: {network.G.graph["temperature_set"]}.
            - Your traits
        """

    question_prompt += f"""
      **Response format (must follow strictly):**

      Reasoning: [Explain your reasoning in 1 paragraph. Mention how your traits influence your decision.]
      Degree: [An **integer only**, e.g., 22]
      Happiness level: [An **integer between 1 and 100**, no text or extra commentary]

      Only provide **one** reasoning, **one** degree, and **one** happiness level. If multiple reasonings exist, combine them into one paragraph.

      Example (format only, not your situation):

      Reasoning: It is 10 degrees outside. I like warmer temperatures, and others do too. I think 24 degrees is a good balance. I felt okay yesterday, but I want to be a bit warmer today.
      Degree: 24
      Happiness level: 90
      [/INST]"""

    try:
        # print("Prompt for node" , agent_id, ":", question_prompt)
        output = get_completion_from_messages(model = model,
                                              tokenizer = tokenizer,
                                              user_prompt = question_prompt)
        # print("Output for node", agent_id, ":", output)
    except Exception as e:
        print(f"{e}\nProgram paused. Retrying after 10s...")
        time.sleep(10)
        output = get_completion_from_messages(model = model,
                                              tokenizer = tokenizer,
                                              user_prompt = question_prompt)

    reasoning = ""
    degree_choice = ""
    happiness_level = ""
    try:
        # Split the string into parts using '\n' as the separator
        parts = output.split('\n')
        # print("parts:", parts)
        # Initialize variables to store the extracted values
        reasoning = ""
        degree_choice = ""

        # Loop through the parts and assign values to the variables
        for part in parts:
          part = part.strip()

          if part.startswith("Reasoning:"):
              reasoning = part[len("Reasoning:"):].strip()

          elif part.startswith("Degree:"):
              match = re.search(r"Degree:\s*(\d+)", part)
              if match:
                  degree_choice = match.group(1)

          elif part.startswith("Happiness level:"):
              match = re.search(r"Happiness level:\s*(\d+)", part)
              if match:
                  happiness_level = match.group(1)

        # # Print the extracted values
        # print("Reasoning before return in get_family_deg_and_happiness:", reasoning)
        # print("Degree choice before return in get_family_deg_and_happiness:", degree_choice)
        # print("Happiness level before return in get_family_deg_and_happiness:", happiness_level)

        # print(reasoning, response)
        save_current_agent_response(curr_day, agent_id, question_prompt, output, reasoning, degree_choice, happiness_level)

    except:
        print("Reasoning, degree or happiness level were not parsed correctly.")
        degree_choice = "Error"
        happiness_level = "Error"
        reasoning = None

    return reasoning, degree_choice, happiness_level

def set_family_degree(network, family_id, members):
  # Previous temp set
  # temp_set = network.G.nodes[family_id]["family-request"]
  # if network.G.graph["current_day"] != 0:
  requested_values = []
  for n in members:
    requested_values.append(network.G.nodes[n]['degree_choice'])

  avg_requested = int(round(np.average(requested_values)))
  network.G.nodes[family_id]["family-request"] = avg_requested

  return avg_requested
  # else:
  #   return temp_set

def save_current_agent_response(curr_day, curr_node, question_prompt, output, reasoning, degree_choice, happiness_level=None, updated_closeness = None):
  simulation_data = {
      "Day": curr_day,
      "Node": curr_node,
      "Prompt": question_prompt,
      "Output": output,
      "Reasoning": reasoning,
      "Degree choice": degree_choice
  }

  file_name = "agents_response_building_level.json"

  if happiness_level is not None:
    simulation_data["Happiness level"] = happiness_level
    file_name = "agents_response_family_level.json"

  if updated_closeness is not None:
    simulation_data["Updated closeness"] = updated_closeness

  # print("Inside save2")
  if my_project.egress is not None:
      # print("Inside save not none")
      try:
          my_project.egress.save_statusdelta(None, simulation_data, file_name, None)
      except Exception as e:
          print("Error occured", e.with_traceback)
  else:
      print("Egress is none, can't save current agent response.")

In [ ]:
# before iteration method
def set_day_and_weather(network, weather_df):
  network.G.graph["current_day"] += 1
  new_degree = int(weather_df.iloc[network.G.graph["current_day"]]["daily_avg_temp"])
  network.G.graph["degree_outside"] = new_degree
  return new_degree

# After iteration data collection methods

# 1. Cost calculation
def calculate_cost(network):
  current_cost = abs(network.G.graph["degree_outside"] - network.G.graph["temperature_set"])
  return current_cost

# 2. Happiness level calculation
def avg_happiness_level(network):
  happiness_level_total = 0

  for n in network.G.nodes():
    happiness_level_total += network.G.nodes[n]["happiness_level"]

  return round(happiness_level_total/network.G.number_of_nodes(), 2)

# 3. Cost/happiness ratio calculation
def cost_happiness_ratio(network):
  return calculate_cost(network)/avg_happiness_level(network)

# 4. A function that saves degree set
# Will use to draw charts
def temperature_set(network):
  return  network.G.graph["temperature_set"]

# 5. Update the last 3 days
def update_last_3_suggestions(network):
  for n in network.G.nodes():
    last3 = network.G.nodes[n]['last_3_suggestions']
    last3.pop(0)
    last3.append(network.G.nodes[n]['degree_choice'])
    network.G.nodes[n]['last_3_suggestions'] = last3

def get_friendship_graph(network):
    """Extracts a subgraph of only 'friend' relationships."""
    return nx.Graph(
        (u, v, data) for u, v, data in network.G.edges(data=True) if data.get('relation') == 'friend'
    )

def average_weighted_degree(network):
    """Calculates the average weighted degree of the friendship graph."""
    G_friend = get_friendship_graph(network)
    return float(np.average([G_friend.degree(u, weight='weight') for u in G_friend.nodes()]))

def average_friendship_weight(network):
    """Computes the average friendship weight."""
    G_friend = get_friendship_graph(network)
    weights = [data['weight'] for _, _, data in G_friend.edges(data=True)]
    return float(np.mean(weights)) if weights else 0.0

def number_of_strong_friendships(network, threshold=3):
    """Counts the number of friendships above a given closeness threshold."""
    G_friend = get_friendship_graph(network)
    return sum(1 for _, _, data in G_friend.edges(data=True) if data['weight'] > threshold)

def clustering_coefficient(network, trials=1000, seed=10):
    """Approximates the average clustering coefficient."""
    G_friend = get_friendship_graph(network)
    return float(nx.algorithms.approximation.average_clustering(G_friend, trials=trials, seed=seed))


# Save data to a dataframe, which is saved to a csv file every iteration. 
# This is used for the regression analyses after the simulation
def save_data_to_df(network, sim_data_df):
  degree_centralities = nx.degree_centrality(network.G)
  closeness_centralities = nx.closeness_centrality(network.G)
  betweenness_centralities = nx.betweenness_centrality(network.G)
  eigenvector_centralities = nx.eigenvector_centrality(network.G)

  rows = []
  for node in network.G.nodes():
    dict_to_add = {
        "node_id": node,
        "degree_choice": network.G.nodes[node]["degree_choice"],
        "happiness_level":  network.G.nodes[node]["happiness_level"],
        "heater_preference": network.G.nodes[node]["heater_preference"],
        "age": network.G.nodes[node]["age"],
        "N2": network.G.nodes[node]["N2"],
        "N5": network.G.nodes[node]["N5"],
        "E3": network.G.nodes[node]["E3"],
        "O3": network.G.nodes[node]["O3"],
        "A3": network.G.nodes[node]["A3"],
        "A4": network.G.nodes[node]["A4"],
        "C6": network.G.nodes[node]["C6"],
        "environmentalism": network.G.nodes[node]["environmentalism"],
        "frugality": network.G.nodes[node]["frugality"],
        "degree_centrality": degree_centralities[node],
        "closeness_centrality": closeness_centralities[node],
        "betweenness_centrality": betweenness_centralities[node],
        "eigenvector_centrality": eigenvector_centralities[node],
        "network_avg_weighted_degree": average_weighted_degree(network),
        "network_avg_friendship_weight": average_friendship_weight(network),
        "network_strong_friendships": number_of_strong_friendships(network),
        "network_degree_outside": network.G.graph["degree_outside"],
        "network_temperature_set": network.G.graph["temperature_set"],
        "network_current_day": network.G.graph["current_day"],
        "network_cost": calculate_cost(network),
        "network_avg_happiness_level": avg_happiness_level(network),
        "network_cost_happ_ratio": cost_happiness_ratio(network)
    }
    rows.append(dict_to_add)

  # Convert list of dictionaries into a DataFrame and concatenate with the existing DataFrame
  sim_data_df = pd.concat([sim_data_df, pd.DataFrame(rows)], ignore_index=True)
  sim_data_df.to_csv(os.path.join(my_project.parent_simulation_dir, ('simulation_data_day_'+ str(network.G.graph["current_day"]) + '_all.csv')))
  # return sim_data_df



In [ ]:
# Step 8: Create or load project

project_name = "energy_sim_1"

my_project = Project()
creation_date = "28/04/2025"
info = "Building energy modeling with LLMs setting the degree"

# Create new project if it doesn't exist already
# The last parameter is optional, if provided, the project is created in that directory
# my_project.create_project(project_name, creation_date, info, "node", "/content/drive/My Drive/Crowd_Related_Work/")
conf_path = os.path.join('/content/drive/My Drive/Crowd_Related_Work/energy_simulation', 'simple_test_conf.yaml')
# my_project.update_conf_with_path(conf_path)

# OR load previous project
# If the project is created in a directory other than the default user directory, the path must be provided as the second parameter
my_project.load_project(project_name, "/content/drive/My Drive/Crowd_Related_Work/")
# conf_path = os.path.join(my_project.project_dir, 'simple_test_conf.yaml')
my_project.update_conf_with_path(conf_path)

In [ ]:
# Execute this code block only if you wish to re-create the family network
# Otherwise, execute the next two blocks if you wish to just update the positive trait distributions
add_name_parameter(my_project.netw.G)
add_trait_parameters(my_project.netw.G, positive_percentage = 50)
print(my_project.netw.G.nodes[0])

# setup families and write to file
# sample_family_sizes 0 to 4 means we will add 0-4 members to each family.
# the representative of the family already exists
# returns families dictionary in the following format: {family_id: [members]}
families = create_family(
    network = my_project.netw.G,
    family_count = my_project.netw.G.number_of_nodes(),
    sample_family_sizes = [0, 1, 2, 3, 4]
)

# write this dictionary to a json file
with open(os.path.join(my_project.project_dir, 'family_members3.json'), 'w') as file:
  json.dump(families, file)

# are all parameters set for new family members or do we need to call any functions here?

# save network file as .json
my_project.egress2 = fd(my_project.project_dir)
my_project.egress2.save_as_json(my_project.netw.G, file_name = "families3.json")

In [ ]:
# If no need to create the network from scratch, load the network file from json
try:
  with open(os.path.join(my_project.project_dir, 'families3.json'), 'r') as file:
    loaded_graph = nx.node_link_graph(json.load(file), edges="links")
  with open(os.path.join(my_project.project_dir, 'family_members3.json'), 'r') as file:
    families = json.load(file)
except Exception as e:
  print(f'Could not load family information: {e}')
  loaded_graph = None
  families = None

# set the loaded network as project's network
my_project.netw.G = loaded_graph

# families' indices should be converted to integer
families = {int(k): v for k, v in families.items()}

In [ ]:
# Update traits function is called here
# Adjust the positive percentage depending on the case being tested
update_traits(my_project.netw.G, positive_percentage = 50)

In [ ]:
# set back crowd's custom sim network parameters
my_project.netw.node_types = ["Family_Rep", "Family_Member"]
my_project.netw.curr_type_nums = my_project.netw.count_node_types()

In [ ]:
# Step 9: Select the execution periods for the custom methods and run the simulation
sim_data_df = pd.DataFrame()

before_iteration_methods = [
    [set_day_and_weather, mid_feb_to_mid_mar_df],
    [decide_degree_for_families, families, model, tokenizer],
    [decide_degree_for_building, families, model, tokenizer]
]
after_iteration_methods = [
    calculate_cost,
    avg_happiness_level,
    cost_happiness_ratio,
    temperature_set,
    update_last_3_suggestions,
    average_weighted_degree,
    average_friendship_weight,
    number_of_strong_friendships,
    clustering_coefficient,
    [save_data_to_df, sim_data_df]
]

# The 0th iteration in Crowd is for setup
# To execute a simulation of 30 days, we provide the iteration number, epochs, as 31
my_project.lib_run_simulation(
    epochs = 31,
    snapshot_period = 1,
    before_iteration_methods = before_iteration_methods,
    after_iteration_methods=after_iteration_methods
)

# save data collected in the dataframe to csv
sim_data_df.to_csv(os.path.join(my_project.parent_simulation_dir, 'simulation_data.csv'))

##### Step 10: Visualization and regression analyses

Code for the regression analyses can be found on *analysis.ipynb* and *analysis_network_level.ipynb* files on the parent folder. 

Following the execution of the simulation with 3 different distributions of the personality traits, we merge the results using Crowd's GUI. The generated charts are provided in Figure 10 of the [paper](https://arxiv.org/abs/2507.09657), which also includes the network visualization, details the implementation methodology and discusses the results.